In [1]:
%matplotlib notebook
%load_ext autoreload
%autoreload 1
!hostname
!pwd

dv001.ib.bridges2.psc.edu
/ocean/projects/asc170022p/mtragoza/mre-pinn/IPMI-2023


In [2]:
import sys, os
import numpy as np
import pandas as pd

sys.path.append('..')
%aimport mre_pinn

sys.path.append('../../param_search')
%aimport param_search
ps = param_search

Using backend: pytorch



# IPMI 2023 cohort experiment

In [3]:
%pwd

'/ocean/projects/asc170022p/mtragoza/mre-pinn/IPMI-2023'

In [41]:
# define the job template and name format

template = '''\
#!/bin/bash
#SBATCH --job-name={job_name}
#SBATCH --account=asc170022p
#SBATCH --partition=GPU-shared
#SBATCH --gres=gpu:v100-32:1
#SBATCH --time=48:00:00
#SBATCH -o %J.stdout
#SBATCH -e %J.stderr
#SBATCH --mail-type=all

hostname
pwd
source activate MRE-PINN

python ../../../train_pino.py \\
    --xarray_dir {xarray_dir} \\
    --frequency {frequency} \\
    --pde_name {pde_name} \\
    --pde_init_weight {pde_init_weight} \\
    --pde_loss_weight {pde_loss_weight} \\
    --pde_warmup_iters {pde_warmup_iters}000 \\
    --pde_step_iters {pde_step_iters}000 \\
    --learning_rate {learning_rate} \\
    --conditional {conditional} \\
    --parallel {parallel} \\
    --save_prefix {job_name}    
'''
name = 'train_{data_name}_{conditional}_{parallel}_{pde_name}_{learning_rate:.0e}_{pde_warmup_iters}k_{pde_step_iters}k'

# define the parameter space

cohort_space = ps.ParamSpace(
    data_name='cohort',
    xarray_dir='../../../data/NAFLD/v3',
    frequency=40,
    pde_name=['helmholtz', 'hetero'],
    pde_init_weight=1e-18,
    pde_loss_weight=1e-16,
    pde_warmup_iters=[10, 20],
    pde_step_iters=[5, 10],
    learning_rate=[1e-4, 5e-5, 2e-5, 1e-5],
) * (
    ps.ParamSpace(
        conditional=[0, 1],
        parallel=0
    ) + ps.ParamSpace(
        conditional=1,
        parallel=1
    )
)

param_space = cohort_space
len(param_space)

96

In [42]:
name.format(**param_space[0])

'train_cohort_0_0_helmholtz_1e-04_10k_5k'

In [43]:
%autoreload
expt_name = '2022-11-28_cohort_init'

#jobs = ps.submit(template, name, param_space, work_dir=expt_name, verbose=True)
#jobs.to_csv(f'{expt_name}.jobs')

100%|██████████| 96/96 [00:00<00:00, 141.49it/s]
[13342305, 13342306, 13342307, 13342308, 13342309, 13342310, 13342311, 13342312, 13342313, 13342314, 13342315, 13342316, 13342317, 13342318, 13342319, 13342320, 13342321, 13342322, 13342323, 13342324, 13342325, 13342326, 13342327, 13342328, 13342329, 13342330, 13342331, 13342332, 13342333, 13342334, 13342335, 13342336, 13342337, 13342338, 13342339, 13342340, 13342341, 13342342, 13342343, 13342344, 13342345, 13342346, 13342347, 13342348, 13342349, 13342350, 13342351, 13342352, 13342353, 13342354, 13342355, 13342356, 13342357, 13342358, 13342359, 13342360, 13342361, 13342362, 13342363, 13342364, 13342365, 13342366, 13342367, 13342368, 13342369, 13342370, 13342371, 13342372, 13342373, 13342374, 13342375, 13342376, 13342377, 13342378, 13342379, 13342380, 13342381, 13342382, 13342383, 13342384, 13342385, 13342386, 13342387, 13342388, 13342389, 13342390, 13342391, 13342392, 13342393, 13342394, 13342395, 13342396, 13342397, 13342398, 13342399, 

In [49]:
status_cols = ['job_name', 'job_state', 'node_id', 'runtime', 'stdout', 'stderr']
status = ps.status(jobs)
status[status_cols]

,job_name,job_state,node_id,runtime,stdout,stderr
job_id,,,,,,
13342305,train_cohort_0_0_helmholtz_1e-04_10k_5k,NaN,NaN,NaN,v011.ib.bridges2.psc.edu\n/ocean/projects/asc1...,Using backend: pytorch\n\nWARNING:py.warnings:...
13342306,train_cohort_1_0_helmholtz_1e-04_10k_5k,RUNNING,v010,11:21:27,v010.ib.bridges2.psc.edu\n/ocean/projects/asc1...,Using backend: pytorch\n\nWARNING:py.warnings:...
13342307,train_cohort_1_1_helmholtz_1e-04_10k_5k,RUNNING,v015,11:15:25,v015.ib.bridges2.psc.edu\n/ocean/projects/asc1...,Using backend: pytorch\n\nWARNING:py.warnings:...
13342308,train_cohort_0_0_helmholtz_5e-05_10k_5k,RUNNING,v014,11:15:25,v014.ib.bridges2.psc.edu\n/ocean/projects/asc1...,Using backend: pytorch\n\nWARNING:py.warnings:...
13342309,train_cohort_1_0_helmholtz_5e-05_10k_5k,RUNNING,v023,4:25:06,v023.ib.bridges2.psc.edu\n/ocean/projects/asc1...,Using backend: pytorch\n\nWARNING:py.warnings:...
...,...,...,...,...,...,...
13342396,train_cohort_1_0_hetero_2e-05_20k_10k,PENDING,(Priority),0:00,NaN,NaN
13342397,train_cohort_1_1_hetero_2e-05_20k_10k,PENDING,(Priority),0:00,NaN,NaN
13342398,train_cohort_0_0_hetero_1e-05_20k_10k,PENDING,(Priority),0:00,NaN,NaN


In [58]:
print('\n'.join(status.iloc[0].stderr.split('\n')[-20:]))

/home/conda/feedstock_root/build_artifacts/pytorch-recipe_1664412099808/work/aten/src/ATen/native/cuda/IndexKernel.cu:91: operator(): block: [176,0,0], thread: [127,0,0] Assertion `index >= -sizes[i] && index < sizes[i] && "index out of bounds"` failed.
Traceback (most recent call last):
  File "/ocean/projects/asc170022p/mtragoza/mre-pinn/IPMI-2023/2022-11-28_cohort_init/train_cohort_0_0_helmholtz_1e-04_10k_5k/../../../train_pino.py", line 15, in <module>
    def train(
  File "/ocean/projects/asc170022p/mtragoza/mre-pinn/mre_pinn/utils.py", line 232, in main
    func(**kwargs)
  File "/ocean/projects/asc170022p/mtragoza/mre-pinn/IPMI-2023/2022-11-28_cohort_init/train_cohort_0_0_helmholtz_1e-04_10k_5k/../../../train_pino.py", line 106, in train
    model.benchmark(100)
  File "/ocean/projects/asc170022p/mtragoza/mre-pinn/mre_pinn/training/pino_training.py", line 212, in benchmark
    inputs, targets, aux_vars = self.data.train_next_batch()
  File "/ocean/projects/asc170022p/mtragoza/m

In [ ]:
metrics = ps.metrics(jobs)

# did all models train to 100k iterations?
#assert (metrics.groupby('job_name')['iteration'].max() == 100e3).all()

# get the final test evaluations
metrics = metrics[metrics.iteration == 100e3]

param_cols = ['pde_name', 'example_id'] # experimental parameters
index_cols = ['variable_name', 'spatial_frequency_bin', 'spatial_region'] # metric identifiers
metric_cols = ['MSAV', 'PSD', 'MAV'] # metric values

metrics = metrics.groupby(param_cols + index_cols, sort=False)[metric_cols].mean()
metrics = metrics.unstack(level=[len(param_cols)])

def metric_map(t):
    metric_name, var_name = t
    new_col_name = f'{var_name}_{metric_name}'
    new_col_name = new_col_name.replace('diff_MSAV', 'pred_MSAE')
    new_col_name = new_col_name.replace('pde_diff_MSAV', 'PDE_MSAE')
    new_col_name = new_col_name.replace('diff_MAV', 'pred_MAD')
    return new_col_name

metrics.columns = [metric_map(t) for t in metrics.columns.to_flat_index()]
metrics

In [ ]:
m = metrics.reset_index()

#fig = ps.plot(
#    m[m.spatial_region == 'all'],
#    x=param_cols,
#    y=['u_pred_MSAE', 'mu_pred_MSAE', 'direct_pred_MSAE', 'fem_pred_MSAE'],
#    height=2.5,
#    width=2.5,
#    legend=False,
#    tight=True
#)

In [ ]:
m.columns

In [ ]:
m['method'] = 'PINN_' + m['pde_name']

direct_m = m[m.pde_name == 'helmholtz'].copy()
direct_m['method'] = 'direct_helmholtz'
for col in m.columns:
    if col.startswith('direct'):
        direct_m[col.replace('direct', 'mu')] = m[col]
        
hh_fem_m = m[m.pde_name == 'helmholtz'].copy()
hh_fem_m['method'] = 'FEM_helmholtz'
for col in m.columns:
    if col.startswith('fem'):
        hh_fem_m[col.replace('fem', 'mu')] = m[col]

ht_fem_m = m[m.pde_name == 'hetero'].copy()
ht_fem_m['method'] = 'FEM_hetero'
for col in m.columns:
    if col.startswith('fem'):
        ht_fem_m[col.replace('fem', 'mu')] = m[col]
        
mm = pd.concat([direct_m, hh_fem_m, ht_fem_m, m])

In [ ]:
mm['mu_pred_MSAE_relative'] = mm['mu_pred_MSAE'] / mm['mu_true_MSAV']
mm['mu_pred_MAD_relative'] = mm['mu_pred_MAD'] / mm['mu_true_MAV']

In [ ]:
%autoreload
import seaborn as sns
import matplotlib.pyplot as plt

colors = ps.results.get_color_palette(10, type='deep', min_val=0)
blue, orange, green, red, purple, brown, pink, gray, yellow, cyan = colors

colors = [blue, yellow, red, cyan, green]
sns.set_palette(colors)
colors = sns.color_palette()
colors

In [ ]:
fig = ps.plot(
    mm[~mm.spatial_region.isin({'0.0', 'all'})],
    x='method',
    y='mu_pred_MAD_relative',
    height=3,
    width=7,
    legend=True,
    tight=True,
    plot_func=ps.results.barplot
)
for i, patch in enumerate(fig.axes[0].patches):
    plt.setp(patch, facecolor=colors[i])

In [ ]:
fig = ps.plot(
    mm[~mm.spatial_region.isin({'0.0', 'all'})],
    x='spatial_region',
    y='mu_pred_MAD_relative',
    hue='method',
    height=3,
    width=7,
    legend=True,
    tight=True,
    plot_func=ps.results.barplot
)

In [ ]:
fig = ps.plot(
    mm[~mm.spatial_region.isin({'0.0', 'all'})],
    x='example_id',
    y='mu_pred_MAD_relative',
    hue='method',
    height=3,
    width=90,
    legend=True,
    tight=True,
    plot_func=ps.results.barplot
)

In [ ]:
fig = ps.plot(
    mm[(mm.spatial_region == 'all') & (mm.spatial_frequency_bin == 'all')],
    x='method',
    y='mu_pred_MSAE_relative',
    height=7,
    width=7,
    legend=True,
    tight=True,
    plot_func=ps.results.barplot
)
fig.axes[0].set_yscale('log')
for i, patch in enumerate(fig.axes[0].patches):
    plt.setp(patch, facecolor=colors[i])

In [ ]:
%autoreload
import mre_pinn

image_names = ['wave image', 'ground truth', 'direct_helmholtz', 'FEM_helmholtz', 'FEM_hetero', 'PINN_helmholtz', 'PINN_hetero']

def plot_image_grid(example_ids):
    n_rows = len(example_ids)
    n_cols = len(image_names)
    ax_width = 1.2
    ax_height = ax_width

    fig, axes, cbar_ax = mre_pinn.visual.subplot_grid(n_rows, n_cols, ax_height, ax_width, space=0, pad=(0.35,0.15,0.15,0.25))
    for row_idx, example_id in enumerate(example_ids):
        example = mre_pinn.data.MREExample.load_xarrays('../data/NAFLD/v3', example_id)
        for col_idx, image_name in enumerate(image_names):
            ax = axes[row_idx,col_idx]
            #ax.text(0.1, 0.1, f'{row_idx}, {col_idx}')
            if col_idx == 0:
                ax.set_ylabel(example_id, fontsize='medium')
            if row_idx == 0:
                ax.set_title(image_name, fontsize='small')
            ax.set_yticks([])
            ax.set_xticks([])

            if image_name == 'wave image':
                array = example.wave[...,0]
                color_kws = mre_pinn.visual.get_color_kws(array)
                color_kws['vmin'] = -color_kws['vmax']
                array = array.real * example.mre_mask.values[...,0]

            elif image_name == 'ground truth':
                array = example.mre[...,0]
                color_kws = mre_pinn.visual.get_color_kws(array)
                color_kws['vmin'] = -color_kws['vmax']
                array = np.abs(array) * example.mre_mask.values[...,0]

            elif image_name == 'direct_helmholtz':
                nc_file = f'2022-11-26_patient_hetero2/train_patient_{example_id}_helmholtz/train_patient_{example_id}_helmholtz_direct.nc'
                array = mre_pinn.data.dataset.load_xarray_file(nc_file).sel(variable='direct_pred')[...,0]
                color_kws = mre_pinn.visual.get_color_kws(array)
                color_kws['vmin'] = -color_kws['vmax']
                array = np.abs(array)

            elif image_name == 'FEM_helmholtz':
                nc_file = f'2022-11-26_patient_hetero2/train_patient_{example_id}_helmholtz/train_patient_{example_id}_helmholtz_fem.nc'
                array = mre_pinn.data.dataset.load_xarray_file(nc_file).sel(variable='fem_pred')[...,0]
                color_kws = mre_pinn.visual.get_color_kws(array)
                color_kws['vmin'] = -color_kws['vmax']
                array = np.abs(array)

            elif image_name == 'FEM_hetero':
                nc_file = f'2022-11-23_patient_init/train_patient_{example_id}_hetero/train_patient_{example_id}_hetero_fem.nc'
                array = mre_pinn.data.dataset.load_xarray_file(nc_file).sel(variable='fem_pred')[...,0]
                color_kws = mre_pinn.visual.get_color_kws(array)
                color_kws['vmin'] = -color_kws['vmax']
                array = np.abs(array)

            elif image_name == 'PINN_helmholtz':
                nc_file = f'2022-11-26_patient_hetero2/train_patient_{example_id}_helmholtz/train_patient_{example_id}_helmholtz_elastogram.nc'
                array = mre_pinn.data.dataset.load_xarray_file(nc_file).sel(variable='mu_pred')[...,0]
                color_kws = mre_pinn.visual.get_color_kws(array)
                color_kws['vmin'] = -color_kws['vmax']
                array = np.abs(array)

            elif image_name == 'PINN_hetero':
                nc_file = f'2022-11-23_patient_init/train_patient_{example_id}_hetero/train_patient_{example_id}_hetero_elastogram.nc'
                array = mre_pinn.data.dataset.load_xarray_file(nc_file).sel(variable='mu_pred')[...,0]
                color_kws = mre_pinn.visual.get_color_kws(array)
                color_kws['vmin'] = -color_kws['vmax']
                array = np.abs(array)

            mre_pinn.visual.imshow(ax, array, **color_kws)

plot_image_grid(['0006', '0020', '0024', '0029'])
plot_image_grid(['0043', '0047', '0126', '0135'])